<a href="https://colab.research.google.com/github/lingelizabeth/senior-research/blob/master/Final_Notebook_TESS_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**TESS Training and Evaluation**
This notebook trains a neural network on analyzed TESS ([Toronto Emotional Speech Set](https://tspace.library.utoronto.ca/handle/1807/24487)) speech data to predict 7 emotion classes. It utilizes a tiered model structure that first trains a Support Vector Machine to determine the possitive/negative affect of a sample, then uses that predicted label as input into a neural net. The final neural net accuracy is ~99% across all classes.

##**Load and clean data**
The inputs are CSV files generated by openSMILE, an audio feature extraction package, and they include statistical measures of audio characteristics for each sample. Specifically, I used the emobase feature set in the openSMILE toolkit.

In [ ]:
from scipy.io import *
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import io
import pandas as pd
import random
from google.colab import files

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from scipy.interpolate import *
from scipy.signal import *

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree, svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from itertools import cycle

import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.utils import to_categorical


In [ ]:
uploaded = files.upload() # upload CSV output of openSMILE run

In [ ]:
# read in CSV to pandas dataframe
df = pd.read_csv(io.StringIO(uploaded['TESSemobase.csv'].decode('utf-8')))

# use only every other line due to openSMILE output format
dropped = [i for i in range(1, df.shape[0], 2)]
df = df.drop(dropped)
df = df.reset_index()

# drop unneccessary columns
new_df = df.drop(['name', 'frameTime', 'index'], axis=1)

In [ ]:
# assigns each row as "positive" or "negative" emotion, based on its true label
positive = [] #1 for yes 0 for negative
for index, row in df.iterrows():
  if(int(row['emotion']) in [1, 2, 3, 8]): 
    positive.append(1)
  else:
    positive.append(0)
    
# append this column to the dataframe for later validation
new_df['positive'] = pd.Series(positive)

In [ ]:
# scale the data to have a mean of 0 and unit variance (standard deviation of 1)
scaler = StandardScaler()
scaled_new_df = scaler.fit_transform(new_df)

# fix indexing and drop labels for scaled dataframe
scaled_new_df = pd.DataFrame(scaled_new_df, index = range(0, scaled_new_df.shape[0]), columns = new_df.columns.values)
scaled_new_df = scaled_new_df.drop(['emotion', 'positive'], axis=1)
scaled_new_df

,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,pcm_intensity_sma_stddev,pcm_intensity_sma_skewness,pcm_intensity_sma_kurtosis,pcm_intensity_sma_quartile1,pcm_intensity_sma_quartile2,pcm_intensity_sma_quartile3,pcm_intensity_sma_iqr1-2,pcm_intensity_sma_iqr2-3,pcm_intensity_sma_iqr1-3,pcm_loudness_sma_max,pcm_loudness_sma_min,pcm_loudness_sma_range,pcm_loudness_sma_maxPos,pcm_loudness_sma_minPos,pcm_loudness_sma_amean,pcm_loudness_sma_linregc1,pcm_loudness_sma_linregc2,pcm_loudness_sma_linregerrA,pcm_loudness_sma_linregerrQ,pcm_loudness_sma_stddev,pcm_loudness_sma_skewness,pcm_loudness_sma_kurtosis,pcm_loudness_sma_quartile1,pcm_loudness_sma_quartile2,pcm_loudness_sma_quartile3,pcm_loudness_sma_iqr1-2,pcm_loudness_sma_iqr2-3,pcm_loudness_sma_iqr1-3,mfcc_sma[1]_max,mfcc_sma[1]_min,...,voiceProb_sma_de_iqr2-3,voiceProb_sma_de_iqr1-3,F0_sma_de_max,F0_sma_de_min,F0_sma_de_range,F0_sma_de_maxPos,F0_sma_de_minPos,F0_sma_de_amean,F0_sma_de_linregc1,F0_sma_de_linregc2,F0_sma_de_linregerrA,F0_sma_de_linregerrQ,F0_sma_de_stddev,F0_sma_de_skewness,F0_sma_de_kurtosis,F0_sma_de_quartile1,F0_sma_de_quartile2,F0_sma_de_quartile3,F0_sma_de_iqr1-2,F0_sma_de_iqr2-3,F0_sma_de_iqr1-3,F0env_sma_de_max,F0env_sma_de_min,F0env_sma_de_range,F0env_sma_de_maxPos,F0env_sma_de_minPos,F0env_sma_de_amean,F0env_sma_de_linregc1,F0env_sma_de_linregc2,F0env_sma_de_linregerrA,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,-0.541741,-0.165073,-0.541741,-1.057407,-0.294749,-0.577746,0.312828,-0.574656,-0.557998,-0.251123,-0.547531,-0.705389,-0.749567,-0.497096,-0.572123,-0.571734,-0.565944,-0.547172,-0.567376,-0.910132,-0.672365,-0.901636,0.176990,-0.324964,-0.714492,0.094803,-0.633960,-0.749242,-0.670725,-0.809863,-0.354460,-0.321672,-0.349050,-0.671702,-0.688638,-0.807095,-0.569326,-0.721913,1.233809,-0.061893,...,-0.308284,-0.055736,-0.049251,-1.196325,0.707471,-0.528177,-1.130025,0.227628,1.225073,-1.394265,1.650490,1.485390,1.416139,-0.574505,-0.953529,-2.177460,0.215175,1.127455,2.204112,1.129989,1.729224,0.962266,-0.771414,1.079540,-0.837986,-0.752221,-0.200990,-0.314810,0.413579,2.062043,1.782280,1.611712,-0.871542,-0.937723,-2.452795,-0.098963,2.087933,2.527921,2.163567,2.582346
1,-0.518678,-0.165073,-0.518678,1.239589,-0.294749,-0.599269,0.284781,-0.579783,-0.573037,-0.251667,-0.559401,-0.183882,-0.144893,-0.691543,-0.693814,-0.582475,-0.668751,-0.533979,-0.572207,-0.839482,-0.672365,-0.829843,1.264389,-0.324964,-0.897759,-0.274842,-0.580240,-0.693622,-0.643493,-0.738372,-0.315970,-0.478831,-1.131762,-0.896461,-0.780494,-0.351022,-0.498314,-0.483820,1.303512,0.470383,...,-0.480155,0.304863,-1.266209,0.734368,-1.111569,-0.513537,0.997203,-1.413950,-0.468141,0.148424,0.130599,-0.478893,-0.390729,-0.590890,-0.909698,-0.615412,0.215175,0.054732,0.640617,0.036728,0.324640,-1.102781,-0.491282,-0.680469,-0.837986,-0.787695,0.065626,0.234802,-0.164297,0.659048,0.066745,0.161252,-1.801217,-1.557251,0.162809,-0.098963,0.390457,-0.193517,0.415559,0.221990
2,-0.550113,-0.165073,-0.550113,1.201306,-0.294749,-0.632922,0.291141,-0.608238,-0.603209,-0.252646,-0.583983,-0.216169,-0.298944,-0.694030,-0.694511,-0.610883,-0.669068,-0.567035,-0.601099,-1.029562,-0.672365,-1.022996,0.079901,-0.324964,-1.069212,0.041443,-0.881485,-0.855368,-0.743304,-0.940641,-0.390068,-0.437317,-1.126474,-1.054938,-0.929957,-0.637010,-0.608084,-0.677319,0.924674,0.244664,...,-0.092635,0.039967,0.489630,-0.750907,0.716006,-0.879542,0.885894,0.994552,0.571235,-0.289956,1.603919,1.386804,1.340847,-0.457939,-0.859792,-2.199055,0.215175,1.180817,2.225727,1.184372,1.772598,0.511672,-1.193386,0.891053,-0.837986,-0.681272,1.1220

## **Binary classification of positive and negative emotions**

My model architecture finding was a significant increase in emotion classification accuracy when first considering a binary classification: **is the sample "positive" or "negative"?** Emotions like happiness, surprise, and neutral were considered positive, while anger, fear, sadness, and disgust were considered negative. 

I use a Support Vector Machine for binary classification for this dataset. The overall accuracy is 98.3%.


In [ ]:
# split into training and validation data
X_train, X_val, y_train, y_val = train_test_split(scaled_new_df, new_df['positive'], test_size = .25, random_state = None)

# Gridsearch to determine best parameters
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear','rbf'], 'gamma':[0.001, 0.01, 0.1, 1]}
test_svm = svm.SVC(gamma='scale')
gridsearchsvm = GridSearchCV(test_svm, param_grid, cv=5)
gridsearchsvm.fit(X_train, y_train)


In [ ]:
#prints best parameters from Grid Search
gridsearchsvm.best_params_

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

In [ ]:
# create SVM classifier
svm = svm.SVC(C=1, gamma=.001, kernel='rbf')
svm.fit(X_train, y_train)
svm_y_predict = svm.predict(X_val)

# evaluate model, accuracy of 99.8%
print(confusion_matrix(y_val, svm_y_predict))
print(accuracy_score(y_val, svm_y_predict))

[[346   0]
 [  1 237]]
0.9982876712328768


### Confusion Matrix for Positive/Negative Classification

![SVM Confusion Matrix](https://drive.google.com/uc?export=view&id=1HwtaH8--zoBwOIaBrqj4bXQkUPRQQ_QM)

In [ ]:
# view which emotion classes are most often misclassified
misclassified = []
for i in range(len(y_predict)):
  if y_predict[i] != list(y_val)[i]:
    misclassified.append(new_df['emotion'][y_val.index[i]])
dict((x,misclassified.count(x)) for x in set(misclassified))

{'1': 7, '3': 1, '6': 2, '8': 9}

In this case, the most error comes from emotion classes 1 and 8, which represent neutral and pleasant surprise, respectively.

## **Classifying Emotions with a Neural Net**
Now, we will predict 7 emotion classes (happy, sad, neutral, fear, anger, disgust, and pleasant surprise) using a neural net, with the audio features *and* the previous positive/negative prediction as input. The final test accuracy is 98.97%.

In [ ]:
# add the svm binary predictions as an input column 
predicted_pos = svm.predict(scaled_new_df)
scaled_new_df['positive'] = pd.Series(predicted_pos)

# split training and testing data
train_data, val_data, train_labels, val_labels = 
  train_test_split(
      scaled_new_df.loc[X_train.index], 
      new_df.loc[y_train.index]['emotion'].astype(int)-1, 
      test_size = .5, 
      random_state = None
  )
print(train_data.shape)

# reshape the data for training 
train_data = np.array(train_data).reshape((train_data.shape[0], train_data.shape[1], 1))#normal_df.drop([' emotion '], axis=1).values.reshape((1403, 989, 1))
train_data.shape

(875, 989)


(875, 989, 1)

In [ ]:
# build model!
model = []
model = tf.keras.Sequential()
model.add(Dense(4, input_shape = train_data[0].shape))
model.add(Dense(4, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(2, activation='relu'))
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(8, activation = tf.nn.softmax)) 
  
sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics =["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 989, 4)            8         
_________________________________________________________________
dense_5 (Dense)              (None, 989, 4)            20        
_________________________________________________________________
dropout_2 (Dropout)          (None, 989, 4)            0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 989, 4)            16        
_________________________________________________________________
dense_6 (Dense)              (None, 989, 2)            10        
_________________________________________________________________
dropout_3 (Dropout)          (None, 989, 2)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1978)              0         
__________

In [ ]:
# train model
y = model.fit(train_data, to_categorical(train_labels), verbose = True, epochs = 120, validation_split = .2) 

Train on 700 samples, validate on 175 samples
Epoch 1/120
700/700 [==============================] - 0s 647us/sample - loss: 2.4999 - acc: 0.1571 - val_loss: 2.1520 - val_acc: 0.1314
Epoch 2/120
700/700 [==============================] - 0s 210us/sample - loss: 2.2886 - acc: 0.1986 - val_loss: 2.0967 - val_acc: 0.1600
Epoch 3/120
700/700 [==============================] - 0s 212us/sample - loss: 2.2549 - acc: 0.1971 - val_loss: 2.0356 - val_acc: 0.2000
Epoch 4/120
700/700 [==============================] - 0s 207us/sample - loss: 2.0760 - acc: 0.2314 - val_loss: 1.9655 - val_acc: 0.2343
Epoch 5/120
700/700 [==============================] - 0s 213us/sample - loss: 2.0003 - acc: 0.2529 - val_loss: 1.8776 - val_acc: 0.3086
Epoch 6/120
700/700 [==============================] - 0s 225us/sample - loss: 1.9854 - acc: 0.2729 - val_loss: 1.7694 - val_acc: 0.4229
Epoch 7/120
700/700 [==============================] - 0s 214us/sample - loss: 1.8279 - acc: 0.3214 - val_loss: 1.6519 - val_acc: 0.

In [ ]:
# Results on VALIDATION data from 2nd train test split
val_data = np.array(val_data).reshape((val_data.shape[0], val_data.shape[1], 1)) #reshape the validation data to input into the model

# predict and print evaluation metrics
y_pred_proba = model.predict(val_data)
print(confusion_matrix(val_labels, np.argmax(y_pred_proba, axis=1))) 
print(accuracy_score(val_labels, np.argmax(y_pred_proba, axis=1)))

[[ 62   0   6   0   0   0   0]
 [  0 131   0   0   0   0   7]
 [  0   0 164   0   0   0   0]
 [  0   0   0 132   0   0   0]
 [  0   0   0   1  88   0   1]
 [  0   0   0   0   0 145   1]
 [  0   2   0   0   2   1 132]]
0.976


In [ ]:
# Results on TEST data from 1st train test split

X_val_w_positive = X_val
X_val_predicted_pos = svm.predict(X_val) 
X_val_w_positive['positive'] = X_val_predicted_pos #add positive column to validation data
X_val_w_positive

# reshape the validation data to input into the model
X_val_w_positive = np.array(X_val_w_positive).reshape((X_val_w_positive.shape[0], X_val_w_positive.shape[1], 1)) 

# predict and print evaluation metrics
y_pred_X_val = model.predict(X_val_w_positive)
print(confusion_matrix(new_df.loc[y_val.index]['emotion'].astype(int)-1, np.argmax(y_pred_X_val, axis=1))) 
print(accuracy_score(new_df.loc[y_val.index]['emotion'].astype(int)-1, np.argmax(y_pred_X_val, axis=1)))

[[ 45   0   0   0   0   0   0]
 [  0 100   0   0   0   0   1]
 [  0   0 103   0   0   0   0]
 [  0   0   0  86   0   0   0]
 [  0   0   0   1  56   0   0]
 [  0   1   0   0   0  98   1]
 [  0   0   0   0   0   2  90]]
0.9897260273972602


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Confusion Matrix for Multiclass Emotion Classification

![Neural Net Confusion Matrix](https://drive.google.com/uc?export=view&id=1z_d66o__bIVyKMkX1Q6DaM-gomVCz8bK)

## **Predicting on Real World Data**
Since the TESS model has relatively high test accuracy, I decided to test it on a short audio clip of my younger sister. The audio was run through openSMILE separately, with the CSV uploaded to this notebook. Below, you can see that the model predicts with 97% certainty that she is "pleasantly surprised".


In [ ]:
# read in CSV and drop unnecessary columns and label
test_data = pd.read_csv(io.StringIO(uploaded['tiffany_emobase.csv'].decode('utf-8')))
test_data = test_data.drop(['id', 'name', 'emotion', 'frameTime'], axis=1)

# predicting positive/negative affect and appending to sample
test_data['positive'] = svm.predict(test_data)
test_data

,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,pcm_intensity_sma_stddev,pcm_intensity_sma_skewness,pcm_intensity_sma_kurtosis,pcm_intensity_sma_quartile1,pcm_intensity_sma_quartile2,pcm_intensity_sma_quartile3,pcm_intensity_sma_iqr1-2,pcm_intensity_sma_iqr2-3,pcm_intensity_sma_iqr1-3,pcm_loudness_sma_max,pcm_loudness_sma_min,pcm_loudness_sma_range,pcm_loudness_sma_maxPos,pcm_loudness_sma_minPos,pcm_loudness_sma_amean,pcm_loudness_sma_linregc1,pcm_loudness_sma_linregc2,pcm_loudness_sma_linregerrA,pcm_loudness_sma_linregerrQ,pcm_loudness_sma_stddev,pcm_loudness_sma_skewness,pcm_loudness_sma_kurtosis,pcm_loudness_sma_quartile1,pcm_loudness_sma_quartile2,pcm_loudness_sma_quartile3,pcm_loudness_sma_iqr1-2,pcm_loudness_sma_iqr2-3,pcm_loudness_sma_iqr1-3,mfcc_sma[1]_max,mfcc_sma[1]_min,...,voiceProb_sma_de_iqr1-3,F0_sma_de_max,F0_sma_de_min,F0_sma_de_range,F0_sma_de_maxPos,F0_sma_de_minPos,F0_sma_de_amean,F0_sma_de_linregc1,F0_sma_de_linregc2,F0_sma_de_linregerrA,F0_sma_de_linregerrQ,F0_sma_de_stddev,F0_sma_de_skewness,F0_sma_de_kurtosis,F0_sma_de_quartile1,F0_sma_de_quartile2,F0_sma_de_quartile3,F0_sma_de_iqr1-2,F0_sma_de_iqr2-3,F0_sma_de_iqr1-3,F0env_sma_de_max,F0env_sma_de_min,F0env_sma_de_range,F0env_sma_de_maxPos,F0env_sma_de_minPos,F0env_sma_de_amean,F0env_sma_de_linregc1,F0env_sma_de_linregc2,F0env_sma_de_linregerrA,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3,positive
0,0.000053,0.0,0.000053,418.0,0.0,0.000004,-1.466152e-08,0.000009,0.000004,6.095482e-11,0.000008,3.383974,15.77883,3.696106e-08,4.373733e-07,0.000003,4.004122e-07,0.000003,0.000003,3.007297,0.0,3.007297,29.0,0.0,0.868165,-0.001806,1.442519,0.428121,0.316218,0.653089,1.035807,3.224921,0.325209,0.641552,1.250469,0.316343,0.608917,0.92526,13.74189,-32.127,...,0.045658,122.2972,-106.5613,228.8584,622.0,161.0,-3.651018e-08,-0.006032,1.918092,19.64751,1043.561,32.32323,0.005968,5.458329,-5.305835,0.0,7.912611,5.305835,7.912611,13.21845,35.24591,-18.96848,54.21439,72.0,166.0,0.584448,-0.002183,1.278758,3.856892,36.71977,6.07297,1.020721,8.196934,-1.54679,0.0,1.977029,1.54679,1.977029,3.523819,0


In [ ]:
# predicting on test data
model.predict(np.array(test_data).reshape(test_data.shape[0], test_data.shape[1], 1))

array([[2.3528503e-07, 4.5681432e-08, 2.7178640e-02, 1.4097279e-07,
        3.5598394e-04, 4.4679781e-04, 1.0515617e-05, 9.7200769e-01]],
      dtype=float32)